In [ ]:
#In this code, I work on the output from QUaltrics which is in XLSX and numeric format. This data covers the responses of 10 online questionnaires
#that participants had to answer. Here are the list of all questionnaires. 
#1. Trier inventory chronic stress (TICS)
#2. Social value orientation (SVO)
#3. Big Five inventory (10 items-BFI 10)
#4. behavioral inhibition/Approach system(BIS/BAS)
#5. behavioral inhibition system 15 items (BIS 15)
#6. Cognitive reflection test(CRT)
#7. Perspective tasking test (from the paper-Soutschek et al 2016)
#8. Morningness-Eveningness Questionnaire (MEQ)
#9. State trait anxiety (STAI)
#10. Social desirability (SDS-17)

#for each questionnaire there is a certain rules of calculations. Based on these roles I try to calculate the answer of particiapnts for each 
# questionnaire and create a proceed data for our final statistical analyses.


In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
from fuzzywuzzy import process


# Load raw data from online questionnaire
raw_onlinefb = pd.read_excel("online_douman_raw.xlsx", engine='openpyxl')


# Clean up the data
raw_onlinefb = raw_onlinefb.drop(0).drop(columns=[
    'EndDate', 'Status', 'IPAddress', 'Finished', 'RecipientLastName', 'RecordedDate', 'ResponseId',
    'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude',
    'LocationLongitude', 'UserLanguage'])

# Keep only completed surveys from anonymous and qr distribution channels
raw_onlinefb = raw_onlinefb[raw_onlinefb['Progress'] == 100]
raw_onlinefb = raw_onlinefb[raw_onlinefb['DistributionChannel'].isin(['anonymous', 'qr'])]

# Drop rows with empty VP codes
raw_onlinefb.dropna(subset=['Q132_1'], inplace=True)


In [ ]:
#Trier inventory chronic stress (TICS)
# TICS QS columns
df_tics = ['Q2_1', 'Q2_2', 'Q2_3', 'Q2_4', 'Q2_5', 'Q111_1', 'Q111_2', 'Q111_3', 'Q111_4', 'Q111_5',
           'Q76_1', 'Q76_2', 'Q76_3', 'Q76_5', 'Q110_1', 'Q110_2', 'Q110_3', 'Q110_4',
           'Q110_5', 'Q77_1', 'Q77_2', 'Q77_3', 'Q77_4', 'Q77_5', 'Q112_1',
           'Q112_2', 'Q112_3', 'Q112_4', 'Q112_5', 'Q78_1', 'Q78_2', 'Q78_3',
           'Q78_4', 'Q78_5', 'Q113_1', 'Q113_2', 'Q113_3', 'Q113_4',
           'Q113_5', 'Q79_1', 'Q79_2', 'Q79_3', 'Q79_4', 'Q114_1', 'Q114_2',
           'Q114_3', 'Q114_4', 'Q114_5', 'Q80_1', 'Q80_3',
           'Q80_4', 'Q80_5', 'Q115_1', 'Q115_2', 'Q115_3', 'Q115_4',
           'Q115_5']

# Control questions
control_qs = ['Q76_4', 'Q79_5', 'Q80_2']
control_1 = [('True' if i == 4 else 'False') for i in raw_onlinefb[control_qs[0]]]
control_2 = [('True' if i == 1 else 'False') for i in raw_onlinefb[control_qs[1]]]
control_3 = [('True' if i == 2 else 'False') for i in raw_onlinefb[control_qs[2]]]

# TICS subgroups
WOL, SO, PP, WD, EDW, LSR, ST, SI, CW = [1, 4, 44, 54, 17, 27, 38, 50], [49, 57, 7, 19, 28, 39], [8, 12, 14, 22, 30, 23, 32, 40, 43], [5,10,13,41,21,37,48,53],[3,47,20,24,35,55],[2,18,31,46],[6,33,15,26,45,52],[29,34,11,42,51,56],[9,16,25,36]

# Convert label of tic cols into numbers
# with showing they belong to which subcategory of tics
tics_num=[]
tics_dict = dict(zip(df_tics, tics_num))
raw_onlinefb = raw_onlinefb.rename(columns={
    'Q2_1': '1_WOL', 'Q2_2': '2_LSR', 'Q2_3': '3_EDW', 'Q2_4': '4_WOL', 'Q2_5': '5_WD',
    'Q111_1': '6_ST', 'Q111_2': '7_SO', 'Q111_3': '8_PP', 'Q111_4': '9_CW', 'Q111_5': '10_WD',
    'Q76_1': '11_SI', 'Q76_2': '12_PP', 'Q76_3': '13_WD', 'Q76_5': '14_PP',
    'Q110_1': '15_ST', 'Q110_2': '16_CW', 'Q110_3': '17_WOL', 'Q110_4': '18_LSR', 'Q110_5': '19_SO',
    'Q77_1': '20_EDW', 'Q77_2': '21_WD', 'Q77_3': '22_PP', 'Q77_4': '23_PP', 'Q77_5': '24_EDW',
    'Q112_1': '25_CW', 'Q112_2': '26_ST', 'Q112_3': '27_WOL', 'Q112_4': '28_SO', 'Q112_5': '29_SI',
    'Q78_1': '30_PP', 'Q78_2': '31_LSR', 'Q78_3': '32_PP', 'Q78_4': '33_ST', 'Q78_5': '34_SI',
    'Q113_1': '35_EDW', 'Q113_2': '36_CW', 'Q113_3': '37_WD', 'Q113_4': '38_WOL', 'Q113_5': '39_SO',
    'Q79_1': '40_PP', 'Q79_2': '41_WD', 'Q79_3': '42_SI', 'Q79_4': '43_PP',
    'Q114_1': '44_WOL', 'Q114_2': '45_ST', 'Q114_3': '46_LSR', 'Q114_4': '47_EDW', 'Q114_5': '48_WD',
    'Q80_1': '49_SO', 'Q80_3': '50_WOL', 'Q80_4': '51_SI', 'Q80_5': '52_ST',
    'Q115_1': '53_WD', 'Q115_2': '54_WOL', 'Q115_3': '55_EDW', 'Q115_4': '56_SI', 'Q115_5': '57_SO'})

raw_onlinefb['TICS'] = raw_onlinefb[tics_num].sum(axis=1)
raw_onlinefb['TICS_control1'] = control_1
raw_onlinefb['TICS_control2'] = control_2
raw_onlinefb['TICS_control3'] = control_3

In [ ]:
#2. Social value orientation (SVO)


# Function to process SVO questions
def process_svo_question(raw_data, question, orientations):
    return [orientations.get(x, 'None') for x in raw_data[question]]

# SVO data preparation
svo_cols = ['Q92_1', 'Q103_1', 'Q102_1', 'Q101_1', 'Q100_1', 'Q99_1', 'Q98_1', 'Q97_1', 'Q96_1']
svo_new_cols = ['svo_1', 'svo_2', 'svo_3', 'svo_4', 'svo_5', 'svo_6', 'svo_7', 'svo_8', 'svo_9']
orientation_orders = [
    {1: 'ind', 2: 'Com', 3: 'pro'},
    {1: 'Com', 2: 'pro', 3: 'ind'},
    {1: 'pro', 2: 'ind', 3: 'Com'}
]

# Process and rename SVO columns
for svo_col, new_col, orientations in zip(svo_cols, svo_new_cols, orientation_orders * 3):
    raw_onlinefb[new_col] = process_svo_question(raw_onlinefb, svo_col, orientations)
    raw_onlinefb = raw_onlinefb.rename(columns={svo_col: new_col})

# Count occurrences of prosocial, individualistic, and competitive orientations
raw_onlinefb["prosocial"] = raw_onlinefb.isin(['pro']).sum(axis='columns')
raw_onlinefb["individualistic"] = raw_onlinefb.isin(['ind']).sum(axis='columns')
raw_onlinefb["competetive"] = raw_onlinefb.isin(['Com']).sum(axis='columns')

# Determine SVO_Final based on the count of orientations
conditions = [
    (raw_onlinefb["prosocial"] >= 6),
    (raw_onlinefb["individualistic"] >= 6),
    (raw_onlinefb["competetive"] >= 6)
]
values = ["prosocial", "individualistic", "competetive"]
raw_onlinefb['SVO_Final'] = np.select(conditions, values)



In [ ]:
#3. behavioral inhibition/Approach system(BIS/BAS)
# Prepare data for analysis of BIS/BAS
bis_bas=['Q75_1','Q75_2','Q75_3','Q75_4','Q75_5','Q75_6','Q116_1',
         'Q116_2','Q116_3',
         'Q116_4','Q116_5','Q116_6',
         'Q117_1','Q117_2','Q117_3','Q117_4',
         'Q117_5','Q117_6','Q118_1','Q118_2','Q118_3','Q118_4','Q118_5',
         'Q118_6']

categories = {
    'BIS_Total': [2, 8, 13, 16, 19, 22, 24],
    'BAS_drive': [3, 9, 12, 21],
    'BAS_FUN': [5, 10, 15, 20],
    'BAS_rewards': [4, 7, 14, 18, 23],
    'Filler': [1, 6, 11, 17]
}

# Create a list of category labels
BIS_BAS_num = [f"{i}_{category}" for i in range(1, 25) for category, items in categories.items() if i in items]

# Create a dictionary to re-label the raw column names from raw data for BIS/BAS test
bisbas_dic = dict(zip(bis_bas, BIS_BAS_num))

# Rename columns in the raw_onlinefb DataFrame
raw_onlinefb = raw_onlinefb.rename(columns=bisbas_dic)

# Reverse scoring for specific columns
reverse_scoring = {'2_BIS_Total': {1: 4, 2: 3, 3: 2, 4: 1}, '22_BIS_Total': {1: 4, 2: 3, 3: 2, 4: 1}}
raw_onlinefb = raw_onlinefb.replace(reverse_scoring)

# Compute the sums for each category
for category, items in categories.items():
    if category != 'Filler':
        raw_onlinefb[f'{category}_qs'] = raw_onlinefb[[f"{i}_{category}" for i in items]].sum(axis=1)

# Prepare BIS_15 questions for data analysis
Bis_15_raw=['Q82_1','Q82_2','Q82_3','Q82_4','Q82_5','Q85_1','Q85_2','Q85_3','Q85_4','Q85_5','Q86_1',
'Q86_2','Q86_3','Q86_4','Q86_5']

bis_15_categories = {
    'BIS-15-NPI': [1, 5, 7, 8, 15],
    'BIS-15-MI': [2, 10, 12, 13, 9],
    'BIS-15-ABI': [14, 6, 4, 3, 11]
}

# Create a list of BIS_15 labels
BIS_15_lis = [f"{i}-{category}" for i in range(1, 16) for category, items in bis_15_categories.items() if i in items]
# Synchronize raw data with edited column names
bis15_dic = dict(zip(Bis_15_raw, BIS_15_lis))

# Rename the BIS-15 columns
raw_onlinefb = raw_onlinefb.rename(columns=bis15_dic)

# Reverse scoring for specific questions
reverse_map = {1: 4, 2: 3, 3: 2, 4: 1}
reverse_qs = ['13-BIS-15-MI', '15-BIS-15-NPI', '1-BIS-15-NPI', '7-BIS-15-NPI', '4-BIS-15-ABI']
for q in reverse_qs:
    raw_onlinefb[q] = raw_onlinefb[q].map(reverse_map)
bis_15_mi=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI']
bis_15_npi=['1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI']
bis_15_abi=['3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
BIS_15_total=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI',
       '1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI',
       '3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
# Calculate scores for BIS-15 subcategories and total
categories = {
    'BIS_15_MI': bis_15_mi,
    'BIS_15_NPI': bis_15_npi,
    'BIS_15_ABI': bis_15_abi,
    'BIS_15_total': BIS_15_total}

for cat, items in categories.items():
    raw_onlinefb[cat] = raw_onlinefb[items].sum(axis=1)


In [ ]:
raw_onlinefb.columns.to_list()

In [ ]:
#4. Big Five inventory (10 items-BFI 10)
# BFI data preparation
BFI_Raw = ['Q135_1', 'Q135_2', 'Q135_3', 'Q135_4', 'Q135_5', 'Q136_1', 'Q136_2', 'Q136_3', 'Q136_4', 'Q136_5']
dimensions = {'N': [4, 9], 'E': [1, 6], 'O': [5, 10], 'A': [2, 7], 'C': [3, 8]}
BFI_list = [f'{i}-BFI_{dim}' for i in range(1, 11) for dim, questions in dimensions.items() if i in questions]

BFI_dic = dict(zip(BFI_Raw, BFI_list))
raw_onlinefb = raw_onlinefb.rename(columns=BFI_dic)

reverse_scores = {'1-BFI_E', '3-BFI_C', '4-BFI_N', '5-BFI_O', '7-BFI_A'}
for col in reverse_scores:
    raw_onlinefb[col] = raw_onlinefb[col].map({1: 5, 2: 4, 3: 3, 4: 2, 5: 1})

for dim, questions in dimensions.items():
    cols = [f'{q}-BFI_{dim}' for q in questions]
    raw_onlinefb[f'BFI_{dim}'] = raw_onlinefb[cols].mean(axis=1)



In [ ]:

#5. Cognitive reflection test(CRT)

crt_raw = ['Q88_1', 'Q89_1', 'Q90_1']
crt_columns = ['CRT_ball', 'CRT_machine', 'CRT_lake']
crt_answers = [['5', '0,05', '5 cent'], ['5', '5 Minuten'], ['47', '47 Tage']]

raw_onlinefb = raw_onlinefb.rename(columns=dict(zip(crt_raw, crt_columns)))

for column, answers in zip(crt_columns, crt_answers):
    raw_onlinefb[column] = raw_onlinefb[column].apply(lambda x: 'True' if x in answers else 'False')


In [ ]:

#6. State trait anxiety (STAI)
# Prepare STAI questions for data analysis
# There are two sets of questions, each containing 20 questions

# # STAI data preparation
stais_raw = [f'Q67_{i}' for i in range(1, 6)] + [f'Q119_{i}' for i in range(1, 6)] + [f'Q120_{i}' for i in range(1, 6)] + [f'Q121_{i}' for i in range(1, 6)] + [f'Q71_{i}' for i in range(1, 6)] + [f'Q122_{i}' for i in range(1, 6)] + [f'Q123_{i}' for i in range(1, 6)] + [f'Q124_{i}' for i in range(1, 6)]
nums = [f'{i}-stai' for i in range(1, 41)]

raw_onlinefb.rename(columns=dict(zip(stais_raw, nums)), inplace=True)

reversed_qs = [f'{i}-stai' for i in [1, 2, 5, 8, 10, 11, 15, 16, 19, 20, 21, 23, 26, 27, 30, 33, 34, 36, 39]]
raw_onlinefb[reversed_qs] = raw_onlinefb[reversed_qs].replace({1: 4, 2: 3, 3: 2, 4: 1})

raw_onlinefb['stai_total'] = raw_onlinefb[reversed_qs].sum(axis=1)


In [ ]:

#7. Social desirability (SDS-17)
# Prepare raw data from SDS 17
raw_sds = ['Q33', 'Q65', 'Q49', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64']

# Generate new column names
sds_num = [f"{i}-SDS17" for i in range(1, 18)]

# Create a dictionary to map raw column names to new column names
sds17_dic = dict(zip(raw_sds, sds_num))

# Rename the columns from raw to new column names
raw_onlinefb.rename(columns=sds17_dic, inplace=True)

# Define SDS column names
sds_cols = sds_num

# Calculate the total score for SDS17 by counting the number of 1's (True) in each row
raw_onlinefb['sds_total'] = (raw_onlinefb[sds_cols] == 1).sum(axis=1)


In [ ]:

#8. Morningness-Eveningness Questionnaire (MEQ)


# Prepare raw data for MEQ
cs = [14, 17, 19, 24, 21]
es = [1, 2, 5, 7, 9, 11]
ec = [10, 15, 18, 20, 22, 23, 25]
cc = [3, 4, 6, 8, 13]
meq_raw_cols = ['Q12', 'Q13', 'Q14', 'Q15', 'Q16']
meq_new_cols = ['meq-1', 'meq-2', 'meq-3', 'meq-4', 'meq-5']
meq_col_mapping = dict(zip(meq_raw_cols, meq_new_cols))

# Rename MEQ columns
raw_onlinefb = raw_onlinefb.rename(columns=meq_col_mapping)

# Reverse the second MEQ question
raw_onlinefb['meq-2'] = raw_onlinefb['meq-2'].map({4:1, 3:2, 2:3, 1:4})

# Calculate MEQ total score
raw_onlinefb['meq_total'] = raw_onlinefb[meq_new_cols].sum(axis=1)

# Prepare E-Scale data
raw_cols = ['Q138_1', 'Q138_2', 'Q138_3', 'Q138_4', 'Q138_5', 'Q140_1', 'Q140_2', 'Q140_3', 'Q140_4', 'Q140_5', 'Q141_1',
            'Q141_2', 'Q141_3', 'Q141_4', 'Q141_5', 'Q142_1', 'Q142_2', 'Q142_3', 'Q142_4', 'Q142_5', 'Q143_1', 'Q143_2', 'Q143_3', 
            'Q143_4', 'Q143_5']

e_scale = [f"{i}-escale_{'cs' if i in cs else 'es' if i in es else 'ec' if i in ec else 'cc'}" for i in range(1, 26)]

escale_dict = dict(zip(raw_cols, e_scale))

# Rename E-Scale columns
raw_onlinefb = raw_onlinefb.rename(columns=escale_dict)

# Reverse the 20th E-Scale question
raw_onlinefb['20-escale_ec'] = raw_onlinefb['20-escale_ec'].map({1:5, 2:4, 3:3, 4:2, 5:1})

# Calculate E-Scale total score
raw_onlinefb['e-scale_total'] = raw_onlinefb[e_scale].sum(axis=1)


In [ ]:
#9. Perspective tasking test (from the paper-Soutschek et al 2016)
#to remove the empty cols of avatar test and bringing all under each other
# i need to add 2 to empty cells in col Q500
# then only for rows of q500 that are 2 and if 1_Q194_First Click is empty 
#remove the cols from 1_Q194_First Click to 1_Q196_First Click

#add 2 to Q500
raw_onlinefb.Q500.fillna(2,inplace=True)
 

# Replacing empty string with np.NaN

raw_onlinefb = raw_onlinefb.replace('', np.nan)
df_left=raw_onlinefb[(~raw_onlinefb['1_Q194_First Click'].isnull())&(raw_onlinefb['Q500']==2)]
#remove the unwanted cols from df_left

df_left.drop(columns=df_left.loc[:,'1_Q196_First Click':'64_Q193'], axis=1,inplace=True)
#creating two separated dfs to get rid of duplicate cols
df_right=raw_onlinefb[(raw_onlinefb['1_Q194_First Click'].isnull())&(raw_onlinefb['Q500']==2)]

#remove the unwanted cols from df_right
df_right.drop(columns=df_right.loc[:,'Q500_1_TEXT':'64_Q175'], axis=1,inplace=True)

#df_left name of cols 
col_df_left=df_left.loc[:,'1_Q194_First Click':'64_Q175'].columns.to_list()
#df_right name of cols
col_df_right=df_right.loc[:,'1_Q196_First Click':'64_Q193'].columns.to_list()
#make dict for both cols names
dict_col_name=dict(zip(col_df_right,col_df_left))  
#rename the df_right specific col names by df_left col names
df_right=df_right.rename(dict_col_name,axis=1)
#add the information which order of perspective they did: left or right
df_left['perspective-order']='left'
df_right['perspective-order']='right'
#encode the right and wrong questions
#for now I coded not answered 'NONE'qs as wrong answered question
#left order 5=stimmt nicht überein 4=stimmt überein 
#right order 4=stimmt nicht überein , 5=stimmt überein 
#questions and correct answers

#question_left_5 as correct answer
ql5c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_left[ql5c]= np.where(df_left[ql5c]==5, 0, 1)
#question_right_4 as correct answer
qr4c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_right[qr4c]= np.where(df_right[qr4c]==4, 0, 1)
#question_left_4 as correct answer
ql4c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_left[ql4c]= np.where(df_left[ql4c]==4, 0, 1)
#question_right_5 as correct answer
qr5c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_right[qr5c]= np.where(df_right[qr5c]==5, 0, 1)
# i need to create four groups 1. self congurent (sc) 2. self incongruent(si) 3.other congruent(oc) 4.other incongruent (oi)
sc=['10_Q175','12_Q175','15_Q175','16_Q175','20_Q175','25_Q175','26_Q175','27_Q175','28_Q175','33_Q175','34_Q175','42_Q175','47_Q175','49_Q175'
,'61_Q175','62_Q175']
si=['1_Q175','7_Q175','14_Q175','18_Q175','21_Q175','31_Q175','36_Q175','38_Q175','41_Q175','46_Q175','51_Q175','56_Q175','59_Q175','60_Q175'
,'63_Q175','64_Q175']
oc=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','11_Q175','17_Q175','24_Q175','29_Q175','32_Q175','37_Q175','44_Q175','45_Q175','50_Q175'
,'54_Q175','58_Q175']
oi=['5_Q175','8_Q175','13_Q175','19_Q175','22_Q175','23_Q175','30_Q175','35_Q175','39_Q175','40_Q175','43_Q175','48_Q175','52_Q175','53_Q175'
,'55_Q175','57_Q175']
# remove trials 3 and 63 due to wrong link of picture for participants before the date 17.10.2022
#append both dfs
df_left.drop('Q500_1_TEXT', inplace=True,axis=1)
df = pd.concat([df_left, df_right], ignore_index=True)

#column for sum of sc
df['self_congurent']= df.apply(lambda row: row[sc].sum(),axis=1)
df['self_congurent']=(16-df['self_congurent'])/16
#column for sum of si
df['self_incongurent']=df.apply(lambda row: row[si].sum(),axis=1)
df['self_incongurent']=(16-df['self_incongurent'])/16
#column for sum of oc
df['other_congurent']=df.apply(lambda row: row[oc].sum(),axis=1)
df['other_congurent']=(16-df['other_congurent'])/16
#column for sum of oi
df['other_incongurent']=df.apply(lambda row: row[oi].sum(),axis=1)
df['other_incongurent']=(16-df['other_incongurent'])/16
#calculate the incongurent! if value is >0 means self incongurent and if <0 other ingurent and 0 means no ingurennt
df['perspective']=(df['self_incongurent']-df['self_congurent'])-(df['other_incongurent']-df['other_congurent'])



df.rename(columns={'StartDate':'Date','Q132_1':'Vp-Code'},inplace=True)


In [ ]:
#Rename columns

df.rename(columns={
'StartDate': 'Date',
'Q132_1': 'Vp-Code'
}, inplace=True)

#Define the columns to be exported for statistical analysis

cols_wanted = [
'Date', 'Duration (in seconds)', 'Vp-Code', 'TICS', 'TICS_control1', 'TICS_control2', 'TICS_control3', 'SVO_Final', 'BIS_Total_qs',
 'BAS_drive_qs','BAS_FUN_qs','BAS_rewards_qs', 'BIS_15_MI', 'BIS_15_NPI', 'BIS_15_ABI',
'BIS_15_total', 'BFI_N', 'BFI_E', 'BFI_O', 'BFI_A', 'BFI_C', 'CRT_ball', 'CRT_machine', 'CRT_lake', 'stai_total', 'sds_total',
'meq_total', 'e-scale_total', 'perspective'
]

#Select the desired columns from the dataframe

df = df[cols_wanted]

#Update the Vp-Code for a specific participant

df.loc[df['Vp-Code'] == 'ANPE07B', 'Vp-Code'] = 'HETO16B'

#Sort the dataframe by Vp-Code

df.sort_values(by='Vp-Code', inplace=True)

#Export the edited dataframe to an Excel file

df.to_excel('df_online_DOM.xlsx', index=False)